In [ ]:
%%html
<script src="http://localhost:8080/firefly/firefly_loader.js"></script>

In [ ]:
from IPython.display import HTML, Javascript, display
import ipywidgets as widgets
from traitlets import *

class ImageViewer(widgets.DOMWidget):
    _view_name = Unicode('ImageViewer').tag(sync=True)
    _view_module = Unicode('firefly_widgets').tag(sync=True)
    GridOn = Bool(True).tag(sync=True)
    SurveyKey = Unicode('k').tag(sync=True)
    
class TableViewer(widgets.DOMWidget):
    _view_name = Unicode('TableViewer').tag(sync=True)
    _view_module = Unicode('firefly_widgets').tag(sync=True)
    pageSize = Int(50).tag(sync=True)
    filters = Unicode().tag(sync=True)
    dataUrl = Unicode().tag(sync=True)

In [ ]:
%%javascript
define('firefly_widgets', ["jupyter-js-widgets"], function(widgets) {
    var seq = 1;

    var ImageViewer = widgets.DOMWidgetView.extend({
        // Render the view.
        render: function() {
            this.req = {
                plotId: 'xxq',
                Type      : 'SERVICE',
                Service   : 'TWOMASS',
                Title     : '2mass from service',
                GridOn     : true,
                SurveyKey  : 'k',
                WorldPt    : '10.68479;41.26906;EQ_J2000',
                SizeInDeg  : '.12',
                AllowImageSelection : true
            };

            this.el.id = `imageViewer-${seq++}`;
            this.model.on('change:GridOn change:SurveyKey', this.redraw, this);
            this.redraw = this.redraw.bind(this);
            setTimeout(this.redraw, 0);
         },
        
        redraw: function() {
            this.req.GridOn = this.model.get('GridOn');
            this.req.SurveyKey = this.model.get('SurveyKey');
            firefly.showImage(this.el.id, this.req);
        },
    });

    var TableViewer = widgets.DOMWidgetView.extend({
        // Render the view.
        render: function() {
            this.req = firefly.util.table.makeIrsaCatalogRequest('wise_allwise_p3as_psd', 'WISE', 'wise_allwise_p3as_psd',
                    {   position: '10.68479;41.26906;EQ_J2000',
                        SearchMethod: 'Cone',
                        radius: 300
                    });

            this.el.id = `TableViewer-${seq++}`;
            this.model.on('change:pageSize change:filters', this.redraw, this);
            this.redraw = this.redraw.bind(this);
            this.tableUpdated = this.tableUpdated.bind(this);
            this.removeListner = firefly.util.addActionListener(['table.new','table.sort','tableUi.update'], this.tableUpdated);
            setTimeout(this.redraw, 0);
         },
        
        redraw: function() {
            this.req.pageSize = this.model.get('pageSize');
            this.req.filters = this.model.get('filters');
            firefly.showTable(this.el.id, this.req);
        },
        
        tableUpdated: function(action, state) {
            var dataUrl = firefly.util.table.getTableSourceUrl(
                            firefly.util.table.getTableUiByTblId(this.req.tbl_id));
            this.model.set('dataUrl', dataUrl);
            if (action.payload.tbl_id === this.req.tbl_id) {
                var o_filters = this.model.get('filters');
                var n_filters = action.payload.request.filters;
                if (o_filters != n_filters) {
//                     this.model.set('filters', n_filters);
                }
            }
            this.touch();
        },
    });

    return {
        ImageViewer: ImageViewer,
        TableViewer: TableViewer
    };


});

In [ ]:
iv = ImageViewer(width='400px', height='400px', GridOn=False)
tv = TableViewer(width='600px', height='400px')

In [ ]:
iv

In [ ]:
tv

In [ ]:
iv.GridOn = True
tv.pageSize=15
tv.filters='ra < 10.7'

In [ ]:
tv.dataUrl

In [ ]:
tv.filters

In [ ]:
from astropy.table import Table
from urllib.request import urlopen

In [ ]:
tab = Table.read(urlopen(tv.dataUrl).read().decode("utf-8"), format='ipac')

In [ ]:
tab